In [11]:
from transformers import pipeline, Conversation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import json

In [12]:
conversation = Conversation("")

json_file_path = "./previous_chats.json"
tfidf_vectorizer_pickle_path = "./tfidf_vectorizer.pickle"
tfidf_matrix_train_pickle_path = "./tfidf_matrix_train.pickle"

In [13]:
def talk_to_cb_primary(test_set_sentence, minimum_score, json_file_path, tfidf_vectorizer_pickle_path, tfidf_matrix_train_pickle_path):
    test_set = (test_set_sentence, "")
    

    with open(tfidf_vectorizer_pickle_path, 'rb') as file:
        tfidf_vectorizer = pickle.load(file)
    with open(tfidf_matrix_train_pickle_path, 'rb') as file:
        tfidf_matrix_train = pickle.load(file)

    tfidf_matrix_test = tfidf_vectorizer.transform([test_set_sentence])

    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
    max_cosine = max(cosine[0])

   
    if max_cosine > minimum_score:
        scores = list(cosine[0])
        sorted_indices = np.argsort(scores)[::-1]
        for index in sorted_indices:
            if scores[index] > minimum_score:
                with open(json_file_path, "r") as sentences_file:
                    reader = json.load(sentences_file)
                    return reader[index-2]["response"], scores[index]

  
    return "live_chat", 0

def train_chat(json_file_path, tfidf_vectorizer_pickle_path, tfidf_matrix_train_pickle_path):
    with open(json_file_path, "r") as sentences_file:
        data = json.load(sentences_file)
        sentences = [" No you.", " No you."] + [row["message"] for row in data]

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)


    with open(tfidf_vectorizer_pickle_path, 'wb') as file:
        pickle.dump(tfidf_vectorizer, file)
    with open(tfidf_matrix_train_pickle_path, 'wb') as file:
        pickle.dump(tfidf_matrix_train, file)

    return tfidf_vectorizer, tfidf_matrix_train

def get_hybrid_response(user_input, threshold=0.7):
        response, confidence = talk_to_cb_primary(
            test_set_sentence=user_input,
            minimum_score=threshold,
            json_file_path=json_file_path,
            tfidf_vectorizer_pickle_path=tfidf_vectorizer_pickle_path,
            tfidf_matrix_train_pickle_path=tfidf_matrix_train_pickle_path
        )
    
        if confidence >= threshold:
            return response
        else:
            conversation.add_user_input(user_input)
            result = chatbot(conversation)
            conversation.mark_processed() 
            return result.generated_responses[-1]

In [14]:
train_chat(json_file_path, tfidf_vectorizer_pickle_path, tfidf_matrix_train_pickle_path)


(TfidfVectorizer(),
 <2366x3253 sparse matrix of type '<class 'numpy.float64'>'
 	with 16377 stored elements in Compressed Sparse Row format>)

In [17]:
while True:
    user_input = input("Вы: ")
    if user_input.lower() == "exit":
        print("Диалог завершен.")
        break
    print("Чатбот:", get_hybrid_response(user_input))

Чатбот: hi there !  it had been a long time !  
Чатбот: fine and you ?
Чатбот: nice !  i'm fine too
Чатбот: i'm a bit skinny, average height, yellow eyes, brown hair... 
Диалог завершен.
